In [37]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
 
def load_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["image_name","throttle", "steer", "brake", "speed"]
	df = pd.read_csv(inputPath, sep=",", header=None, names=cols)
 
	# return the data frame
	return df

In [38]:
def process_attributes(df, train, test):
	# initialize the column names of the continuous data
	continuous = ["throttle", "steer", "brake", "speed"]
 
	# performin min-max scaling each continuous feature column to
	# the range [0, 1]
	cs = MinMaxScaler()
	trainContinuous = cs.fit_transform(train[continuous])
	testContinuous = cs.transform(test[continuous])
 
	# construct our training and testing data points by concatenating
	# the categorical features with the continuous features
	trainX = np.hstack([trainContinuous])
	testX = np.hstack([testContinuous])
 
	# return the concatenated training and testing data
	return (trainX, testX)

In [39]:
def load_images(df, inputPath):
	images = []
 
	for i in df.index.values:
		imagePath = os.path.sep.join([inputPath, str(df.at[i,"image_name"])])
		print(imagePath)
		image = cv2.imread(imagePath)
		image  = cv2.pyrDown(image)
	
		image  = cv2.pyrDown(image)
		#outputImage = np.zeros((1280, 720, 3), dtype="uint8")
		#outputImage[0:720, 0:1280] = image[0:720, 0:1280]
		# add the tiled image to our set of images the network will be
		# trained on
		images.append(image)
 
	# return our set of images
	return np.array(images)
    

In [40]:
# import the necessary packages
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
 
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
 
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
 
	# return our model
	return model

In [41]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1
 
	# define the model input
	inputs = Input(shape=inputShape)
 
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs
 
		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
    # flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)
 
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
 
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
 
	# construct the CNN
	model = Model(inputs, x)
 
	# return the CNN
	return model

In [42]:
# import the necessary packages
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import locale
import os

In [43]:


base_path = "E:\carla\carla\_out"
print("[INFO] loading attributes...")
inputPath = os.path.sep.join([base_path, "record.csv"])
print("input path", inputPath)
df = load_attributes(inputPath)
 



[INFO] loading attributes...
input path E:\carla\carla\_out\record.csv


In [44]:
df.head(5)

,image_name,throttle,steer,brake,speed
0,00106017.png,0.403869,-0.029071,0.0,28.975677
1,00106020.png,0.377253,-0.002236,0.0,29.034535
2,00106021.png,0.388633,-0.003561,0.0,29.045440
3,00106022.png,0.383001,-0.008354,0.0,29.040004
4,00106064.png,0.385309,-0.019794,0.0,29.040379


In [45]:
# range [0, 1]
print("[INFO] loading images...")
images = load_images(df, base_path)
images = images / 255.0

[INFO] loading images...
E:\carla\carla\_out\00106017.png
E:\carla\carla\_out\00106020.png
E:\carla\carla\_out\00106021.png
E:\carla\carla\_out\00106022.png
E:\carla\carla\_out\00106064.png
E:\carla\carla\_out\00106098.png
E:\carla\carla\_out\00106177.png
E:\carla\carla\_out\00106178.png
E:\carla\carla\_out\00106266.png
E:\carla\carla\_out\00106267.png
E:\carla\carla\_out\00106311.png
E:\carla\carla\_out\00106393.png
E:\carla\carla\_out\00106394.png
E:\carla\carla\_out\00106469.png
E:\carla\carla\_out\00106470.png
E:\carla\carla\_out\00106516.png
E:\carla\carla\_out\00106595.png
E:\carla\carla\_out\00106596.png
E:\carla\carla\_out\00106680.png
E:\carla\carla\_out\00106681.png
E:\carla\carla\_out\00106753.png
E:\carla\carla\_out\00106797.png
E:\carla\carla\_out\00106798.png
E:\carla\carla\_out\00106839.png
E:\carla\carla\_out\00106877.png
E:\carla\carla\_out\00106954.png
E:\carla\carla\_out\00106988.png
E:\carla\carla\_out\00106989.png
E:\carla\carla\_out\00107021.png
E:\carla\carla\_ou

E:\carla\carla\_out\00115688.png
E:\carla\carla\_out\00115733.png
E:\carla\carla\_out\00115775.png
E:\carla\carla\_out\00115816.png
E:\carla\carla\_out\00115893.png
E:\carla\carla\_out\00115933.png
E:\carla\carla\_out\00115934.png
E:\carla\carla\_out\00115972.png
E:\carla\carla\_out\00116013.png
E:\carla\carla\_out\00116094.png
E:\carla\carla\_out\00116133.png
E:\carla\carla\_out\00116134.png
E:\carla\carla\_out\00116173.png
E:\carla\carla\_out\00116212.png
E:\carla\carla\_out\00116286.png
E:\carla\carla\_out\00116323.png
E:\carla\carla\_out\00116324.png
E:\carla\carla\_out\00116359.png
E:\carla\carla\_out\00116393.png
E:\carla\carla\_out\00116466.png
E:\carla\carla\_out\00116502.png
E:\carla\carla\_out\00116503.png
E:\carla\carla\_out\00116540.png
E:\carla\carla\_out\00116573.png
E:\carla\carla\_out\00116645.png
E:\carla\carla\_out\00116683.png
E:\carla\carla\_out\00116684.png
E:\carla\carla\_out\00116726.png
E:\carla\carla\_out\00116767.png
E:\carla\carla\_out\00116812.png
E:\carla\c

E:\carla\carla\_out\00125465.png
E:\carla\carla\_out\00125507.png
E:\carla\carla\_out\00125508.png
E:\carla\carla\_out\00125547.png
E:\carla\carla\_out\00125586.png
E:\carla\carla\_out\00125669.png
E:\carla\carla\_out\00125711.png
E:\carla\carla\_out\00125712.png
E:\carla\carla\_out\00125753.png
E:\carla\carla\_out\00125791.png
E:\carla\carla\_out\00125866.png
E:\carla\carla\_out\00125900.png
E:\carla\carla\_out\00125901.png
E:\carla\carla\_out\00125937.png
E:\carla\carla\_out\00125973.png
E:\carla\carla\_out\00126054.png
E:\carla\carla\_out\00126093.png
E:\carla\carla\_out\00126094.png
E:\carla\carla\_out\00126129.png
E:\carla\carla\_out\00126164.png
E:\carla\carla\_out\00126235.png
E:\carla\carla\_out\00126276.png
E:\carla\carla\_out\00126277.png
E:\carla\carla\_out\00126315.png
E:\carla\carla\_out\00126356.png
E:\carla\carla\_out\00126390.png
E:\carla\carla\_out\00126464.png
E:\carla\carla\_out\00126507.png
E:\carla\carla\_out\00126548.png
E:\carla\carla\_out\00126549.png
E:\carla\c

E:\carla\carla\_out\00136279.png
E:\carla\carla\_out\00136323.png
E:\carla\carla\_out\00136324.png
E:\carla\carla\_out\00136370.png
E:\carla\carla\_out\00136414.png
E:\carla\carla\_out\00136506.png
E:\carla\carla\_out\00136555.png
E:\carla\carla\_out\00136559.png
E:\carla\carla\_out\00136599.png
E:\carla\carla\_out\00136642.png
E:\carla\carla\_out\00136734.png
E:\carla\carla\_out\00136777.png
E:\carla\carla\_out\00136778.png
E:\carla\carla\_out\00136824.png
E:\carla\carla\_out\00136870.png
E:\carla\carla\_out\00136961.png
E:\carla\carla\_out\00137009.png
E:\carla\carla\_out\00137010.png
E:\carla\carla\_out\00137053.png
E:\carla\carla\_out\00137101.png
E:\carla\carla\_out\00137151.png
E:\carla\carla\_out\00137237.png
E:\carla\carla\_out\00137286.png
E:\carla\carla\_out\00137339.png
E:\carla\carla\_out\00137340.png
E:\carla\carla\_out\00137388.png
E:\carla\carla\_out\00137436.png
E:\carla\carla\_out\00137529.png
E:\carla\carla\_out\00137578.png
E:\carla\carla\_out\00137579.png
E:\carla\c

E:\carla\carla\_out\00145896.png
E:\carla\carla\_out\00145966.png
E:\carla\carla\_out\00146005.png
E:\carla\carla\_out\00146006.png
E:\carla\carla\_out\00146039.png
E:\carla\carla\_out\00146070.png
E:\carla\carla\_out\00146144.png
E:\carla\carla\_out\00146175.png
E:\carla\carla\_out\00146176.png
E:\carla\carla\_out\00146208.png
E:\carla\carla\_out\00146248.png
E:\carla\carla\_out\00146316.png
E:\carla\carla\_out\00146349.png
E:\carla\carla\_out\00146350.png
E:\carla\carla\_out\00146385.png
E:\carla\carla\_out\00146415.png
E:\carla\carla\_out\00146483.png
E:\carla\carla\_out\00146522.png
E:\carla\carla\_out\00146557.png
E:\carla\carla\_out\00146558.png
E:\carla\carla\_out\00146592.png
E:\carla\carla\_out\00146626.png
E:\carla\carla\_out\00146661.png
E:\carla\carla\_out\00146730.png
E:\carla\carla\_out\00146764.png
E:\carla\carla\_out\00146765.png
E:\carla\carla\_out\00146800.png
E:\carla\carla\_out\00146834.png
E:\carla\carla\_out\00146899.png
E:\carla\carla\_out\00146935.png
E:\carla\c

E:\carla\carla\_out\00155077.png
E:\carla\carla\_out\00155110.png
E:\carla\carla\_out\00155111.png
E:\carla\carla\_out\00155143.png
E:\carla\carla\_out\00155178.png
E:\carla\carla\_out\00155250.png
E:\carla\carla\_out\00155283.png
E:\carla\carla\_out\00155284.png
E:\carla\carla\_out\00155317.png
E:\carla\carla\_out\00155351.png
E:\carla\carla\_out\00155420.png
E:\carla\carla\_out\00155458.png
E:\carla\carla\_out\00155494.png
E:\carla\carla\_out\00155495.png
E:\carla\carla\_out\00155528.png
E:\carla\carla\_out\00155559.png
E:\carla\carla\_out\00155591.png
E:\carla\carla\_out\00155656.png
E:\carla\carla\_out\00155692.png
E:\carla\carla\_out\00155693.png
E:\carla\carla\_out\00155722.png
E:\carla\carla\_out\00155759.png
E:\carla\carla\_out\00155833.png
E:\carla\carla\_out\00155869.png
E:\carla\carla\_out\00155870.png
E:\carla\carla\_out\00155902.png
E:\carla\carla\_out\00155937.png
E:\carla\carla\_out\00156002.png
E:\carla\carla\_out\00156035.png


In [46]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("[INFO] processing data...")
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
 
trainY = trainAttrX["steer"]
testY = testAttrX["steer"]

(trainAttrX, testAttrX) = process_attributes(df,
	trainAttrX, testAttrX)


[INFO] processing data...


In [47]:
import tensorflow as tf

In [48]:
# create the MLP and CNN models
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(320, 180, 3, regress=False)
 
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])
 
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
 
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value
model = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError())


C:\Users\Eric\miniconda3\envs\tf-gpu\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [49]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
model

Num GPUs Available:  1


In [50]:
# train the model
print("[INFO] training model...")
model.fit(
	[trainAttrX, trainImagesX], trainY,
	validation_data=([testAttrX, testImagesX], testY),
	epochs=500, batch_size=16)
 


[INFO] training model...
Epoch 1/500
61/61 [==============================] - 3s 41ms/step - loss: 0.2020 - val_loss: 0.0686
Epoch 2/500
61/61 [==============================] - 2s 37ms/step - loss: 0.0185 - val_loss: 0.2835
Epoch 3/500
61/61 [==============================] - 2s 36ms/step - loss: 0.0108 - val_loss: 0.0161
Epoch 4/500
61/61 [==============================] - 2s 35ms/step - loss: 0.0073 - val_loss: 8.8875e-04
Epoch 5/500
61/61 [==============================] - 2s 35ms/step - loss: 0.0049 - val_loss: 7.8198e-04
Epoch 6/500
61/61 [==============================] - 2s 34ms/step - loss: 0.0049 - val_loss: 7.3833e-04
Epoch 7/500
61/61 [==============================] - 2s 34ms/step - loss: 0.0027 - val_loss: 7.1785e-04
Epoch 8/500
61/61 [==============================] - 2s 33ms/step - loss: 0.0026 - val_loss: 7.2331e-04
Epoch 9/500
61/61 [==============================] - 2s 34ms/step - loss: 0.0019 - val_loss: 7.1694e-04
Epoch 10/500
61/61 [==============================]

Epoch 77/500
61/61 [==============================] - 2s 36ms/step - loss: 5.8256e-04 - val_loss: 7.2839e-04
Epoch 78/500
61/61 [==============================] - 2s 37ms/step - loss: 5.9918e-04 - val_loss: 7.2572e-04
Epoch 79/500
61/61 [==============================] - 2s 35ms/step - loss: 6.5678e-04 - val_loss: 7.2516e-04
Epoch 80/500
61/61 [==============================] - 2s 34ms/step - loss: 5.6291e-04 - val_loss: 7.2295e-04
Epoch 81/500
61/61 [==============================] - 2s 36ms/step - loss: 6.0091e-04 - val_loss: 7.2498e-04
Epoch 82/500
61/61 [==============================] - 2s 36ms/step - loss: 5.7362e-04 - val_loss: 7.2726e-04
Epoch 83/500
61/61 [==============================] - 2s 35ms/step - loss: 6.1412e-04 - val_loss: 7.2763e-04
Epoch 84/500
61/61 [==============================] - 2s 35ms/step - loss: 5.7141e-04 - val_loss: 7.2435e-04
Epoch 85/500
61/61 [==============================] - 2s 34ms/step - loss: 5.8234e-04 - val_loss: 7.2700e-04
Epoch 86/500
61/61 

61/61 [==============================] - 2s 35ms/step - loss: 5.6911e-04 - val_loss: 7.2672e-04
Epoch 152/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3853e-04 - val_loss: 7.2510e-04
Epoch 153/500
61/61 [==============================] - 2s 36ms/step - loss: 5.4901e-04 - val_loss: 7.2437e-04
Epoch 154/500
61/61 [==============================] - 2s 35ms/step - loss: 5.5183e-04 - val_loss: 7.2727e-04
Epoch 155/500
61/61 [==============================] - 2s 36ms/step - loss: 5.5510e-04 - val_loss: 7.2478e-04
Epoch 156/500
61/61 [==============================] - 2s 34ms/step - loss: 5.6863e-04 - val_loss: 7.2651e-04
Epoch 157/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4173e-04 - val_loss: 7.2963e-04
Epoch 158/500
61/61 [==============================] - 2s 35ms/step - loss: 5.6316e-04 - val_loss: 7.2441e-04
Epoch 159/500
61/61 [==============================] - 2s 35ms/step - loss: 6.1273e-04 - val_loss: 7.2375e-04
Epoch 160/500
61/61 [===

61/61 [==============================] - 2s 33ms/step - loss: 5.5467e-04 - val_loss: 7.2194e-04
Epoch 226/500
61/61 [==============================] - 2s 34ms/step - loss: 5.5722e-04 - val_loss: 7.2335e-04
Epoch 227/500
61/61 [==============================] - 2s 36ms/step - loss: 5.5356e-04 - val_loss: 7.2469e-04
Epoch 228/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3160e-04 - val_loss: 7.2549e-04
Epoch 229/500
61/61 [==============================] - 2s 36ms/step - loss: 5.6503e-04 - val_loss: 7.2456e-04
Epoch 230/500
61/61 [==============================] - 2s 36ms/step - loss: 5.4092e-04 - val_loss: 7.2290e-04
Epoch 231/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4351e-04 - val_loss: 7.2225e-04
Epoch 232/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3865e-04 - val_loss: 7.2224e-04
Epoch 233/500
61/61 [==============================] - 2s 35ms/step - loss: 5.2985e-04 - val_loss: 7.2422e-04
Epoch 234/500
61/61 [===

61/61 [==============================] - 2s 35ms/step - loss: 5.4726e-04 - val_loss: 7.2484e-04
Epoch 300/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3501e-04 - val_loss: 7.2308e-04
Epoch 301/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4344e-04 - val_loss: 7.2354e-04
Epoch 302/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4946e-04 - val_loss: 7.2179e-04
Epoch 303/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3077e-04 - val_loss: 7.2257e-04
Epoch 304/500
61/61 [==============================] - 2s 34ms/step - loss: 5.2879e-04 - val_loss: 7.2263e-04
Epoch 305/500
61/61 [==============================] - 2s 34ms/step - loss: 5.6368e-04 - val_loss: 7.2483e-04
Epoch 306/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4138e-04 - val_loss: 7.2470e-04
Epoch 307/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3257e-04 - val_loss: 7.2535e-04
Epoch 308/500
61/61 [===

61/61 [==============================] - 2s 35ms/step - loss: 5.3924e-04 - val_loss: 7.2325e-04
Epoch 374/500
61/61 [==============================] - 2s 33ms/step - loss: 5.4142e-04 - val_loss: 7.2298e-04
Epoch 375/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3538e-04 - val_loss: 7.2221e-04
Epoch 376/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3529e-04 - val_loss: 7.2199e-04
Epoch 377/500
61/61 [==============================] - 2s 34ms/step - loss: 5.2642e-04 - val_loss: 7.2071e-04
Epoch 378/500
61/61 [==============================] - 2s 35ms/step - loss: 5.2994e-04 - val_loss: 7.2265e-04
Epoch 379/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3108e-04 - val_loss: 7.2555e-04
Epoch 380/500
61/61 [==============================] - 2s 34ms/step - loss: 5.4189e-04 - val_loss: 7.2221e-04
Epoch 381/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3542e-04 - val_loss: 7.2530e-04
Epoch 382/500
61/61 [===

61/61 [==============================] - 2s 36ms/step - loss: 5.3223e-04 - val_loss: 7.2254e-04
Epoch 448/500
61/61 [==============================] - 2s 36ms/step - loss: 5.3307e-04 - val_loss: 7.2091e-04
Epoch 449/500
61/61 [==============================] - 2s 36ms/step - loss: 5.2164e-04 - val_loss: 7.2214e-04
Epoch 450/500
61/61 [==============================] - 2s 35ms/step - loss: 5.2966e-04 - val_loss: 7.2140e-04
Epoch 451/500
61/61 [==============================] - 2s 34ms/step - loss: 5.3225e-04 - val_loss: 7.2343e-04
Epoch 452/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3531e-04 - val_loss: 7.2189e-04
Epoch 453/500
61/61 [==============================] - 2s 36ms/step - loss: 5.3218e-04 - val_loss: 7.2260e-04
Epoch 454/500
61/61 [==============================] - 2s 35ms/step - loss: 5.3445e-04 - val_loss: 7.2324e-04
Epoch 455/500
61/61 [==============================] - 2s 35ms/step - loss: 5.4047e-04 - val_loss: 7.2273e-04
Epoch 456/500
61/61 [===

In [51]:
# make predictions on the testing data
print("[INFO] predicting steer angle...")
preds = model.predict([testAttrX, testImagesX])

diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 

from sklearn.metrics import mean_squared_error
mse= mean_squared_error(testY,preds.flatten())

mse

[INFO] predicting steer angle...


0.0007235909798150368

In [52]:
model.save("e:\carla\carla\_out\savedmodel.mdl")

INFO:tensorflow:Assets written to: e:\carla\carla\_out\savedmodel.mdl\assets


In [56]:
from tensorflow import keras
reconstructed_model = keras.models.load_model("e:\carla\carla\_out\savedmodel.mdl")

preds2 = reconstructed_model.predict([testAttrX, testImagesX])